this project is used to analyze the best map bp when you queue in the faceit ladder, the browswer lobby king provided some ideas behind this project, however that browser extension is not very detailed and complete.

I aim to develop a more effective way to select the best map.

In [46]:
#import essential libraries
import requests
import pandas as pd
import numpy as np
import pprint
from tabulate import tabulate

In [47]:
#api key and name 
api_key = '9444122f-110e-4f4e-81fb-5162e1c70cac'
api_name = 'Key_FhNKALzMPk'

games_url = f'https://open.faceit.com/data/v4/matches/{input("please enter the faceit match id that you want to analyze")}'
headers = {"Authorization": f"Bearer {api_key}"}

response = requests.get(games_url, headers=headers)
data = response.json()

# print(data)

team_1_roster_playerr_id = []
team_2_roster_playerr_id = []

#get all players id 
team_1_roster = data['teams']['faction1']['roster']
team_2_roster = data['teams']['faction2']['roster']

for player in team_1_roster:
    team_1_roster_playerr_id.append(player['player_id'])
    
for player in team_2_roster:
    team_2_roster_playerr_id.append(player['player_id'])

print(team_1_roster_playerr_id)

['d59c9a03-5a13-47b7-9f6e-e8b7b35c255c', '16b9f05b-3f20-4440-8844-a9eeeb391e85', '8b8086b7-5bcb-4c94-8c47-08bfa4640661', 'a20d1adb-03b1-4b3a-ba40-48ddbf48c031', 'a9160f17-f82a-479d-a8a5-32638f155872']


In [48]:
df = pd.DataFrame(columns = ['player_id', 'match_id', 'map', 'win/loss', 'adr','kd_ratio','enemies_flashed_per_round','flashes_per_round_in_a_match','entry_entry_rate','match_entry_win_rate','sniper_kill_rate_per_match','utility_usage_per_round','utility_damage'])

#next i will retrive the match history of each player

player_match_history = {}

for player_id in team_1_roster_playerr_id:
    print('analyzing player: ', player_id)
    
    player_url = f"https://open.faceit.com/data/v4/players/{player_id}/history?limit=10"
    response = requests.get(player_url, headers=headers).json()

    matches = []
    for i in response['items']:
        matches.append(f"https://open.faceit.com/data/v4/matches/{i['match_id']}/stats")
    
    player_match_history[player_id] = matches


    for match in matches:
        response = requests.get(match, headers=headers).json()

        round_stats = response['rounds'][0]['round_stats']
        
        map = round_stats['Map']
        
        score = round_stats['Score']
        
        winner = round_stats['Winner']
        
        team_1_roster_playerr_id = []
        
        for i in response['rounds'][0]['teams'][0]['players']:
            team_1_roster_playerr_id.append(i['player_id'])
        if winner == response['rounds'][0]['teams'][0]['team_id'] and player_id in team_1_roster_playerr_id:
            win_loss = 1
        else:
            win_loss = 0
        
        for team in response['rounds'][0]['teams']:
            for player in team['players']:
                if player_id == player['player_id']:
                    player_stats = player['player_stats']
        
        adr = player_stats['ADR']
        
        kd_ratio = player_stats['K/D Ratio']
        
        enemies_flashed_per_round = player_stats['Enemies Flashed per Round in a Match']
        
        flashes_per_round_in_a_match = player_stats['Flashes per Round in a Match']
        
        entry_entry_rate = player_stats['Match Entry Rate'] 
        
        match_entry_win_rate = player_stats['Match Entry Success Rate']
        
        sniper_kill_rate_per_match = player_stats['Sniper Kill Rate per Match']
        
        utility_usage_per_round = player_stats['Utility Usage per Round']
        
        utility_damage = player_stats['Utility Damage']
        

    #create a dictionary

        stats = {'player_id':player_id,'match_id':matches[0],'map':map,'win/loss':win_loss, 'adr':adr,'kd_ratio':kd_ratio,'enemies_flashed_per_round':enemies_flashed_per_round,'flashes_per_round_in_a_match':flashes_per_round_in_a_match,'entry_entry_rate':entry_entry_rate,'match_entry_win_rate':match_entry_win_rate,'sniper_kill_rate_per_match':sniper_kill_rate_per_match,'utility_usage_per_round':utility_usage_per_round,'utility_damage':utility_damage}
        
        df.loc[len(df)] = stats


analyzing player:  d59c9a03-5a13-47b7-9f6e-e8b7b35c255c
analyzing player:  16b9f05b-3f20-4440-8844-a9eeeb391e85
analyzing player:  8b8086b7-5bcb-4c94-8c47-08bfa4640661
analyzing player:  a20d1adb-03b1-4b3a-ba40-48ddbf48c031
analyzing player:  a9160f17-f82a-479d-a8a5-32638f155872


In [49]:


df.to_csv('output.csv', index=False) 
